In [1]:
import numpy as np

In [2]:
np.array([1,3,3,7]).shape

(4,)

In [3]:
a = np.array([[1,2,3],[4,5,6]])
a

array([[1, 2, 3],
       [4, 5, 6]])

In [4]:
b = np.array([2,3])

In [5]:
b.shape

(2,)

In [6]:
b.dot(a)

array([14, 19, 24])

In [7]:
np.multiply(b,a)

ValueError: operands could not be broadcast together with shapes (2,) (2,3) 

In [ ]:
b

In [8]:
a.T

array([[1, 4],
       [2, 5],
       [3, 6]])

In [9]:
a.T.dot(b)

array([14, 19, 24])

In [10]:
np.repeat(b,3).reshape(a.shape) * a * wesa

NameError: name 'wesa' is not defined

In [12]:
wesa = np.array([1,2,3])

In [13]:
wesa

array([1, 2, 3])

In [14]:
wesa.dot(a.T * b)

array([28, 96])

In [15]:
np.random.randn(10)

array([ 2.09590074,  0.72679098, -0.07952178, -2.02445839,  1.62350581,
       -0.68985993, -1.45816087,  2.24317027,  0.55676165,  0.34564668])

In [16]:
b

array([2, 3])

In [17]:
a

array([[1, 2, 3],
       [4, 5, 6]])

In [19]:
a.dot(wesa)

array([14, 32])

In [78]:
# -*- coding: utf-8 -*-

import numpy as np
from scipy import special
from sklearn.base import ClassifierMixin, BaseEstimator
from sklearn.datasets import make_classification

# Используйте scipy.special для вычисления численно неустойчивых функций
# https://docs.scipy.org/doc/scipy/reference/special.html#module-scipy.special

def lossf(w, X, y, l1, l2):
    """
    Вычисление функции потерь.

    :param w: numpy.array размера  (M,) dtype = np.float
    :param X: numpy.array размера  (N, M), dtype = np.float
    :param y: numpy.array размера  (N,), dtype = np.int
    :param l1: float, l1 коэффициент регуляризатора 
    :param l2: float, l2 коэффициент регуляризатора
    :return: float, value of loss function
    """
    N, M = X.shape
    lossf = np.sum(np.logaddexp(0, np.exp(w.dot(-X.T * y)))) + l1 * np.sum(abs(w)) + l2 * np.sum(w * w)
    return lossf

def gradf(w, X, y, l1, l2):
    """
    Вычисление градиента функции потерь.

    :param w: numpy.array размера  (M,), dtype = np.float
    :param X: numpy.array размера  (N, M), dtype = np.float
    :param y: numpy.array размера  (N,), dtype = np.int
    :param l1: float, l1 коэффициент регуляризатора
    :param l2: float, l2 коэффициент регуляризатора
    :return: numpy.array размера  (M,), dtype = np.float, gradient vector d lossf / dw
    """
    gradw = np.sum(special.expit(w.dot(-X.T * y)) * ((-X.T * y)), axis=1) + l1 * (np.sign(w)) + 2 * l2 * w
    return gradw

class LR(ClassifierMixin, BaseEstimator):
    def __init__(self, lr=1, l1=1e-4, l2=1e-4, num_iter=1000, verbose=0):
        self.l1 = l1
        self.l2 = l2
        self.w = None
        self.lr = lr
        self.verbose = verbose
        self.num_iter = num_iter

    def fit(self, X, y):
        """
        Обучение логистической регрессии.
        Настраивает self.w коэффициенты модели.

        Если self.verbose == True, то выводите значение
        функции потерь на итерациях метода оптимизации.

        :param X: numpy.array размера  (N, M), dtype = np.float
        :param y: numpy.array размера  (N,), dtype = np.int
        :return: self
        """
        n, d = X.shape
        self.w = np.random.randn(d)
        precision = 1e-6
        cur_loss = lossf(self.w, X, y, self.l1, self.l2)
        prev_loss = cur_loss + 1e10
        if self.verbose:
            print('First iter loss = ', cur_loss)

        iter_num = 0
        while(abs(cur_loss - prev_loss) > precision and iter_num <= self.num_iter):
            iter_num += 1
            self.w -= self.lr * gradf(self.w, X, y, self.l1, self.l2)
            prev_loss = cur_loss
            cur_loss = lossf(self.w, X, y, self.l1, self.l2)
            if self.verbose:
                print('current_loss at iter number {} equals to {}'.format(iter_num, cur_loss) )

        return self

    def predict_proba(self, X):
        """
        Предсказание вероятности принадлежности объекта к классу 1.
        Возвращает np.array размера (N,) чисел в отрезке от 0 до 1.

        :param X: numpy.array размера  (N, M), dtype = np.float
        :return: numpy.array размера  (N,), dtype = np.int
        """
        # Вычислите вероятности принадлежности каждого
        # объекта из X к положительному классу, используйте
        # эту функцию для реализации LR.predict
        # 1. / (1. + np.exp(-self.decision_function(X)));
        probs = 1. / (1. + np.exp(-X.dot(self.w)))
        return probs

    def predict(self, X):
        """
        Предсказание класса для объекта.
        Возвращает np.array размера (N,) элементов 1 или -1.

        :param X: numpy.array размера  (N, M), dtype = np.float
        :return:  numpy.array размера  (N,), dtype = np.int
        """
        # Вычислите предсказания для каждого объекта из X
        predicts = np.sign(X.dot(self.w))
        return predicts

def test_work():
    print "Start test"
    X, y = make_classification(n_features=100, n_samples=1000)
    y = 2 * (y - 0.5)

    try:
        clf = LR(lr=1, l1=1e-4, l2=1e-4, num_iter=1000, verbose=0)
    except Exception:
        assert False, "Создание модели завершается с ошибкой"
        return

    try:
        clf = clf.fit(X, y)
    except Exception:
        assert False, "Обучение модели завершается с ошибкой"
        return

    assert isinstance(lossf(clf.w, X, y, 1e-3, 1e-3), float), "Функция потерь должна быть скалярной и иметь тип np.float"
    assert gradf(clf.w, X, y, 1e-3, 1e-3).shape == (100,), "Размерность градиента должна совпадать с числом параметров"
    assert gradf(clf.w, X, y, 1e-3, 1e-3).dtype == np.float, "Вектор градиента, должен состоять из элементов типа np.float"
    assert clf.predict(X).shape == (1000,), "Размер вектора предсказаний, должен совпадать с количеством объектов"
    assert np.min(clf.predict_proba(X)) >= 0, "Вероятности должны быть не меньше, чем 0"
    assert np.max(clf.predict_proba(X)) <= 1, "Вероятности должны быть не больше, чем 1"
    assert len(set(clf.predict(X))) == 2, "Метод предсказывает больше чем 2 класса на двух классовой задаче"
    print "End tests"

test_work()


Start test
End tests


/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:23: RuntimeWarning: overflow encountered in exp
/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:70: RuntimeWarning: invalid value encountered in double_scalars
/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:92: RuntimeWarning: overflow encountered in exp


In [80]:
X, y = make_classification(n_features=100, n_samples=1000)
y = 2 * (y - 0.5)
clf = LR(lr=1, l1=1e-4, l2=1e-4, num_iter=1000, verbose=0)




In [81]:
clf = clf.fit(X, y)



/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:23: RuntimeWarning: overflow encountered in exp
/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:70: RuntimeWarning: invalid value encountered in double_scalars


In [85]:
-X.dot(clf.w)

array([ -3.71169084e+02,   1.15518216e+03,  -1.62723427e+03,
        -1.94834171e+02,  -3.79153128e+02,  -7.80670678e+02,
         2.36064772e+02,   9.00116503e+02,   7.88894456e+02,
        -8.69026057e+02,   3.43289752e+02,   1.31952752e+03,
        -2.86184290e+02,  -9.74960945e+02,  -5.14827974e+02,
         1.04668619e+03,   1.83502347e+02,  -2.88867509e+02,
        -1.35542494e+03,   6.17471812e+02,   6.76549661e+02,
        -1.95326202e+03,  -6.44249789e+02,   1.11650919e+02,
         3.40935668e+02,  -3.32212261e+02,   9.34450116e+02,
         4.61425685e+01,   7.89767358e+02,  -7.12195055e+02,
        -1.08096420e+03,   1.13782675e+03,  -5.89651804e+02,
        -1.55054210e+03,   1.54955444e+03,  -1.13517846e+03,
        -1.28892746e+03,  -1.06307061e+03,  -6.36299586e+02,
        -4.94797752e+02,  -1.25913486e+02,  -1.30889324e+02,
        -3.67271350e+02,  -5.89025732e+02,   7.61602408e+02,
         3.31682126e+02,   2.99821923e+02,   2.79031136e+02,
        -1.34454460e+03,

array([  18.02074837,   40.31238383,   20.85456203,   -5.23084625,
        -29.98637658,   -7.89284188,  149.97579291,   26.14499875,
        -15.22832235,   42.84318633,   25.27944839,  -14.12831451,
        -41.26634995,   32.23624846,    1.56901983,   10.4543113 ,
        -28.01229405,  -36.82974699,    4.52147268,   30.21301272,
         41.6859287 ,   -3.56899546,  -17.39511037,  -22.35398347,
         -6.8312643 ,   24.77795076,    3.4008095 ,   27.73318567,
         68.2755404 ,   -7.06606795,  -23.27776657,   -2.31112189,
          7.46768932,   34.33779617,  -30.53791033,   45.20029576,
        -23.67280673,  -34.72692265,    5.89716403,   -3.75067594,
         25.21799546,   23.85922046,   47.44473292,   14.57823687,
        -40.62989773,    1.54004109,    2.69001673,  -27.30687273,
        -70.85535087,  -60.16124074,  -47.93772351,  -46.82199075,
        -16.51684201,  -38.96864043,    4.9122052 ,  -10.85788299,
        -23.32569701,   14.21143045,  -97.38873584,   -7.15813

In [87]:
clf.predict_proba(X)

/Users/semenfedotov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:92: RuntimeWarning: overflow encountered in exp


array([  1.00000000e+000,   0.00000000e+000,   1.00000000e+000,
         1.00000000e+000,   1.00000000e+000,   1.00000000e+000,
         3.00865338e-103,   0.00000000e+000,   0.00000000e+000,
         1.00000000e+000,   8.14995422e-150,   0.00000000e+000,
         1.00000000e+000,   1.00000000e+000,   1.00000000e+000,
         0.00000000e+000,   2.02275488e-080,   1.00000000e+000,
         1.00000000e+000,   6.84540588e-269,   1.50735523e-294,
         1.00000000e+000,   1.00000000e+000,   3.24057568e-049,
         8.58065685e-149,   1.00000000e+000,   0.00000000e+000,
         9.13139470e-021,   0.00000000e+000,   1.00000000e+000,
         1.00000000e+000,   0.00000000e+000,   1.00000000e+000,
         1.00000000e+000,   0.00000000e+000,   1.00000000e+000,
         1.00000000e+000,   1.00000000e+000,   1.00000000e+000,
         1.00000000e+000,   1.00000000e+000,   1.00000000e+000,
         1.00000000e+000,   1.00000000e+000,   0.00000000e+000,
         8.95948045e-145,   6.15167582e-

In [37]:
y.shape

(1000,)

In [65]:
gradf(clf.w, X, y, clf.l1, clf.l2).shape

(100, 1000)

In [64]:
clf.w - gradf(clf.w, X, y, clf.l1, clf.l2)

ValueError: operands could not be broadcast together with shapes (100,) (100,1000) 

In [61]:
len(gradf(clf.w, X, y, clf.l1, clf.l2))

100

In [54]:
a.T * b

array([[ 2, 12],
       [ 4, 15],
       [ 6, 18]])

In [55]:
a.T

array([[1, 4],
       [2, 5],
       [3, 6]])

In [56]:
b * a.T

array([[ 2, 12],
       [ 4, 15],
       [ 6, 18]])

In [77]:
clf.predict(X)

array([-1.,  1.,  1., -1., -1., -1.,  1.,  1.,  1., -1., -1.,  1., -1.,
        1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,
        1.,  1., -1.,  1., -1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,
        1.,  1., -1.,  1., -1., -1.,  1., -1., -1., -1., -1., -1., -1.,
        1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,
       -1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,
       -1., -1., -1.,  1., -1.,  1.,  1., -1.,  1., -1., -1.,  1.,  1.,
       -1., -1., -1., -1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1.,
        1., -1.,  1., -1., -1., -1., -1.,  1.,  1.,  1., -1., -1.,  1.,
        1.,  1., -1., -1.,  1.,  1., -1.,  1.,  1., -1., -1.,  1., -1.,
       -1.,  1., -1.,  1., -1.,  1., -1., -1., -1.,  1., -1., -1., -1.,
        1., -1.,  1., -1., -1., -1.,  1., -1., -1.,  1.,  1.,  1., -1.,
        1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        1.,  1.,  1., -1., -1., -1., -1.,  1., -1., -1.,  1., -1